### tables needed </br>
•account_derived_metadata</br>
•account_info</br>
•user_info</br>
•user_phones</br>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import random
import json
import psycopg2
import datetime as dt
import dateutil.parser
from dateutil.relativedelta import relativedelta
import re
import csv
from nameparser import HumanName
from os import listdir
from ast import literal_eval
from functools import reduce
from IPython.core.interactiveshell import InteractiveShell

# database variables
cred     = json.load(open('credential.json'))
database = 'gemini'
port     = '55432'
user     = cred[database]['username']
password = cred[database]['password']
localhost = '127.0.0.1'

def query(sql):
    conn = psycopg2.connect(host=localhost,
        port=port,
        user=user,
        password=password,
        dbname=database)
    return pd.read_sql_query(sql, conn)

def query_file(sql_file):
    with open(sql_file, "r") as f:
        sql = f.read()
    return query(sql)

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ModuleNotFoundError: No module named 'nameparser'

In [2]:
# query('''
# select *
# from user_info
# where birthdate in ('1989-05-22', '1992-02-03') and
# legal_last_name ilike ('thomas')
# ''')

In [9]:
# raw_bus = pd.read_csv('2019-08-27_business.csv')
# raw_per = pd.read_csv('2019-08-27_person.csv')

raw_bus = pd.read_excel('~/Documents/2019-08-27.xls',sheet_name='business')
raw_per = pd.read_excel('~/Documents/2019-08-27.xls',sheet_name='people')

In [10]:
raw_per['dob'] = pd.to_datetime(raw_per['dob'])

In [11]:
dates = []

for n in raw_per['dob']:
    if n.year > dt.datetime.now().year:
        dates.append(n - relativedelta(years = 100))
    else:
        dates.append(n)

raw_per['dob'] = dates

In [12]:
raw_bus.head()
raw_bus.shape
raw_per.head()
raw_per.shape

(513, 18)

In [13]:
# def name_check(row):
#     fn = row['first_name']
#     ln = row['last_name']
    
#     results = query(
#     '''
#     select
#         exchange_user_id
#     from
#         user_info
#     where
#         legal_last_name = '{0}'
#     ''').format(ln)
    
#     if results.shape[0] == 0:
#         return None
#     if results.shape[0] != 0:
#         return results['exchange_user_id']

In [14]:
raw_per['last_name'] = [n.replace("'", "''") for n in raw_per['last_name']]
raw_bus['business_name'] = [n.replace("'", '') for n in raw_bus['business_name']]

In [15]:
def last_name_search(df, column):
    names_list = [n.replace("'", "''") for n in df[column]]
    names_list = ["'"+n.lower()+"'" for n in df[column]]
    names = ', '.join(names_list)
    query_str = '''
    select birthdate, lower(legal_last_name) legal_last_name, exchange_user_id
    from user_info
    where lower(legal_last_name) in ({})
    '''.format(names)
    results = query(query_str)
    
#     dobs = []
    
#     for t in raw_per['dob']:
#         try:
#             dobs.append(t.strftime('%Y-%m-%d'))
#         except:
#             continue
    
#     dobs = ["'"+t+"'" for t in dobs]
#     dobs = ', '.join(dobs)
    
#     results = results[results['birthdate'].isin(raw_per['dob'])]
    
    return results

In [16]:
def ln_dob_match(gem_df, fincen_df):
    match_data = []
    for gem_ln, gem_dob, gem_id in zip(gem_df['legal_last_name'], gem_df['birthdate'], gem_df['exchange_user_id']):
        for i, row in fincen_df.iterrows():
            try:
                if (gem_ln == row['last_name']) & (gem_dob == row['dob']):
                    match_data.append([gem_ln, gem_dob, gem_id, row['tracking_number']])
            except:
                continue
    return pd.DataFrame(match_data, columns = ['gem_last_name', 'gem_dob', 'user_id', 'tracking_number'])

In [17]:
def street_search(df):
    
    zips_list = ["'"+str(int(n))+"'" for n in df[df['zip'].notnull()]['zip']]
    streets = ["'"+s.lower()+"'" for s in df[df['street'].notnull()]['street']]
    
    zips = ', '.join(zips_list)
    streets = ', '.join(streets)
    
    results = query(
    '''
    select zip, lower(street) street, exchange_user_id
    from user_info
    where lower(street) in ({0})
    '''.format(streets)
    )
    
    results = results[results['zip'].isin(zips_list)]
    
    return results

In [18]:
def street_zip_match(gem_df, fincen_df):
    match_data = []
    for gem_street, gem_zip, gem_id in zip(gem_df['street'], gem_df['zip'], gem_df['exchange_user_id']):
        for i, row in fincen_df.iterrows():
            try:
                if (gem_street == row['street']) & (gem_zip == str(int(row['zip']))):
                    match_data.append([gem_street, gem_zip, gem_id, row['tracking_number']])
            except:
                continue
    return pd.DataFrame(match_data, columns = ['gem_street', 'gem_ziop', 'user_id', 'tracking_number'])

In [19]:
def bus_name_search(df, column):
    
    names_list = ["'"+n.lower()+"'" for n in df[column]]
    names = ', '.join(names_list)
    results = query(
    '''
    select lower(user_or_account_name) user_or_account_name, exchange_account_id
    from account_derived_metadata
    where
        is_institutional is true and
        lower(user_or_account_name) in ({})
    '''.format(names)
    )
    return results

In [20]:
lns = last_name_search(raw_per, 'last_name')
lns_dob_match = ln_dob_match(lns, raw_per)

streets_zips = street_search(raw_per)
street_zips_match = street_zip_match(streets_zips, raw_per)

NameError: name 'query' is not defined

In [21]:
bus_names = bus_name_search(raw_bus, 'business_name')
bus_streets_zips = street_search(raw_bus)
bus_streets_zips_match = street_zip_match(bus_streets_zips, raw_bus)

NameError: name 'query' is not defined

In [22]:
lns_dob_match.head()
lns_dob_match.shape
street_zips_match.head()
street_zips_match.shape

NameError: name 'lns_dob_match' is not defined

In [23]:
bus_names.head()
bus_names.shape
bus_streets_zips_match.head()
bus_streets_zips_match.shape

NameError: name 'bus_names' is not defined

### name matching, street/zip combo, dob